In [45]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
import warnings 
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
from PIL import Image

In [57]:
import os
file_path='C:\\Users\\NanduCn\\jupter1\\train-scene classification'
train=pd.read_csv(os.path.join(file_path,'train.csv'))
test=pd.read_csv(os.path.join(file_path,'test.csv'))


In [58]:
train.head()

,image_name,label
0,0.jpg,0
1,1.jpg,4
2,2.jpg,5
3,4.jpg,0
4,7.jpg,4


In [104]:
temp = []
for img_name in train.image_name:
    img_path = os.path.join(file_path, 'train', img_name)
    img = imread(img_path)
    img = imresize(img, (32, 32))
    img = img.astype('float32') # this will help us in later stage
    temp.append(img)

train_x = np.stack(temp)

In [111]:

for img_name in test.image_name:
    img_path = os.path.join('C:\\Users\\NanduCn\\jupter1\\train-scene classification', 'train', img_name)
    imag = imread(img_path)
    

In [114]:
temp = []
imag = imresize(imag, (32, 32))
temp.append(imag.astype('float32'))
test_x = np.stack(temp)

In [116]:
train_x.shape

(17034, 32, 32, 3)

In [117]:
train_x = train_x / 255.
test_x = test_x / 255.

In [118]:
train.label.value_counts(normalize=True)

3    0.178290
2    0.173594
5    0.169250
4    0.163438
1    0.161148
0    0.154280
Name: label, dtype: float64

In [120]:
import keras
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
train_y = lb.fit_transform(train.label)
train_y = keras.utils.np_utils.to_categorical(train_y)

In [121]:
input_num_units = (32, 32, 3)
hidden_num_units = 500
output_num_units = 3

epochs = 5
batch_size = 128

In [122]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, InputLayer

In [123]:
model = Sequential([
  InputLayer(input_shape=input_num_units),
  Flatten(),
  Dense(units=hidden_num_units, activation='relu'),
  Dense(units=output_num_units, activation='softmax'),
])

In [124]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 500)               1536500   
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 1503      
Total params: 1,538,003
Trainable params: 1,538,003
Non-trainable params: 0
_________________________________________________________________


In [125]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=batch_size,epochs=epochs,verbose=1)


ValueError: Error when checking target: expected dense_8 to have shape (3,) but got array with shape (6,)